In [1]:
!pip install git+https://github.com/openai/swarm.git
!pip install openai
!pip install firecrawl-py

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-56uiyx12
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-56uiyx12
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 20.2 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=b6a62ca9d1573

In [2]:
import os
from firecrawl import FirecrawlApp
from swarm import Agent, Swarm
from openai import OpenAI
import pandas as pd
import numpy as np

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "FirecrawlApp.ExtractParams" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [3]:
api = OpenAI(api_key="") #OpenAI Token Here

In [4]:
client = Swarm(api)

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/XCai777/AI_Republic_Bootcamp/refs/heads/main/AI_First_Day_6/ai_first_sales_data_sales.csv')

In [12]:
data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['weekday'] = data['date'].dt.weekday

def clean_currency(value):
    """
    Removes non-numeric characters and converts to integer.
    """
    if isinstance(value, str):  # Ensure it's a string
        # Remove currency symbols and commas
        cleaned_value = value.replace('₱', '').replace(',', '')
        # Convert to integer
        return int(float(cleaned_value))  # Handles decimals
    return value  # Return value if it's already numeric

# Apply the function to the 'revenue' column
data['ad spend'] = data['ad spend'].apply(clean_currency)
data['revenue'] = data['revenue'].apply(clean_currency)

# **HAND OFF**


In [13]:
def analyze_cost_effectiveness():
      # Calculate cost per transaction and revenue per spend
      data['cost_per_transaction'] = data['ad spend'] / (data['transactions'] + 1)
      data['revenue_per_spend'] = data['revenue'] / (data['ad spend'] + 1)

      # Identify the top-performing traffic sources
      grouped = data.groupby('source').agg({
            'ad spend': 'sum',
            'transactions': 'sum',
            'revenue': 'sum'
      })
      grouped['revenue_per_spend'] = grouped['revenue'] / (grouped['ad spend'] + 1)
      return grouped.sort_values('revenue_per_spend', ascending=False)

def optimize_roi():
      # Identify campaigns with low ROI
      data['roi'] = data['revenue'] / (data['ad spend'] + 1)
      low_roi = data[data['roi'] < 1]
      return low_roi[['source', 'medium', 'ad spend', 'revenue', 'roi']]

def flag_wastage():
      # Highlight campaigns with high spend but low conversions
      wastage = data[(data['ad spend'] > 1000) & (data['transactions'] < 10)]
      return wastage[['source', 'medium', 'ad spend', 'transactions']]

def suggest_scaling():
      # Recommend scaling for campaigns with high ROI
      high_roi = data[data['roi'] > 3]
      return high_roi[['source', 'medium', 'ad spend', 'revenue', 'roi']]

# **AGENTS**

In [14]:
cost_analyst_agent = Agent(
    name="User Interface Agent",
    model="gpt-4o-mini",
    instructions="You are a cost analyst agent speciliazed in analyzing cost-effectiveness of different traffic sources and marketing campaigns.",
    functions=[analyze_cost_effectiveness],
)

roi_optimizer_agent = Agent(
    name="ROI Optimizer Agent",
    model="gpt-4o-mini",
    instructions="You are an ROI Optimizer Agent that calculates ROI per source/medium and identify underperforming campaigns.",
    functions=[optimize_roi],
)

wastage_agent = Agent(
    name="Wastage Analyst Agent",
    model="gpt-4o-mini",
    instructions="You are a wastage analyst agent that identifies campaigns with high spend but low conversion",
    functions=[flag_wastage],
)

scaling_strategist_agent = Agent(
    name="Scaling Strategist Agent",
    model="gpt-4o-mini",
    instructions="You are a scaling strategist agent that determines high-performing sources, devices, or campaigns that can be scaled up profitably.",
    functions=[suggest_scaling],
    )

# **Output**

In [37]:
# Run the Cost Analyst Agent
cost_analyst_response = client.run(
      agent=cost_analyst_agent,
      messages=[{"role": "user", "content": "Based on the data, I want to know the cost-effectiveness."}]
  )

print("\nCost Analysis Results:")
print(cost_analyst_response.messages[-1]["content"])


Cost Analysis Results:
Here is the cost-effectiveness analysis of various traffic sources based on ad spend, transactions, revenue, and revenue per spend:

| Source             | Ad Spend     | Transactions | Revenue       | Revenue per Spend |
|--------------------|--------------|--------------|---------------|--------------------|
| (direct)           | 2,758,336,907| 197,467      | 1,256,781,676 | 0.455630           |
| google             | 6,490,566,537| 388,169      | 2,406,178,484 | 0.370719           |
| facebook           | 7,141,533,169| 377,909      | 2,465,492,458 | 0.345233           |
| tiktok             | 2,308,870,038| 55,397       | 346,623,075   | 0.150127           |
| cityads            | 1,616,344,923| 11,904       | 75,351,293    | 0.046618           |
| instagram          | 2,004,995,201| 14,501       | 86,724,053    | 0.043254           |
| promo              | 657,223,313  | 4,395        | 28,281,240    | 0.043031           |
| co-promo           | 510,268,396

In [36]:
# Run the ROI Optimizer Agent
roi_optimizer_response = client.run(
      agent=roi_optimizer_agent,
      messages=[{"role": "user", "content": "Based on the data, I want to know how may ROI can be optimized."}]
  )

print("\nROI Optimizer Results:")
print(roi_optimizer_response.messages[-1]["content"])


ROI Optimizer Results:
The ROI data reveals several sources and mediums with varying levels of performance. Here are some insights based on the provided information:

1. **Underperforming Campaigns**:
   - Many campaigns have a low ROI, indicating that the revenue generated is significantly lower than the ad spend.
   - Examples of underperforming sources include:
     - TikTok (cpa) with an ROI of 0.106786.
     - Direct traffic campaigns with ROIs as low as 0.006380.
     - Co-promotion via email yielding an ROI of 0.089354.

2. **Better Performing Campaigns**:
   - The "facebook organic" channel has a relatively higher ROI of 0.760383.
   - Other channels like "google cpc" have a moderate ROI of 0.655382, indicating potential for optimization without significant changes.

3. **Potential Areas for Optimization**:
   - Focus on campaigns with a ROI below 0.2 to analyze why they are underperforming and identify improvements.
   - Assess the performance of channels that are underperfor

In [27]:
# Run the Wastage Analyst Agent
wastage_response = client.run(
      agent=wastage_agent,
      messages=[{"role": "user", "content": "Based on the data, I want to know the wastage of each campaigns."}]
  )

print("\nWastage Results:")
print(wastage_response.messages[-1]["content"])


Wastage Results:
Here is a summary of the campaigns with their corresponding ad spend and transactions:

| Source      | Medium | Ad Spend | Transactions |
|-------------|--------|----------|--------------|
| tiktok      | cpa    | 369185   | 6            |
| instagram   | cpc    | 372718   | 6            |
| instagram   | cpc    | 833607   | 4            |
| (direct)    | (none) | 470848   | 4            |
| tiktok      | cpa    | 526308   | 9            |
| (direct)    | (none) | 491731   | 4            |
| co-promo    | email  | 433598   | 1            |
| google      | cpc    | 198570   | 1            |
| (direct)    | (none) | 684341   | 1            |
| tiktok      | cpa    | 523643   | 4            |

This data provides insight into campaign performance, with ad spends that appear high but yield low conversions in some cases, such as those with only 1 or 4 transactions. Would you like a more detailed analysis or a specific focus on campaigns that have the highest spend with the

In [28]:
# Run the Scaling Strategist Agent
scaling_strategist_response = client.run(
      agent=scaling_strategist_agent,
      messages=[{"role": "user", "content": "Based on the data, I want to know how I can scale my business."}]
  )

print("\nScaling Strategist Results:")
print(scaling_strategist_response.messages[-1]["content"])


Scaling Strategist Results:
To determine how to scale your business effectively, I analyzed the data based on sources, mediums, ad spend, revenue, and return on investment (ROI). Here are some highlighted insights based on high ROI and revenue potential:

1. **High-Performing Ad Spend**:
   - **Facebook CPC**: The campaigns with CPC (Cost Per Click) on Facebook performed particularly well, such as one with an ad spend of $162,954 generating $988,383 in revenue (ROI of 6.07). This indicates significant potential for scaling in this area.
   - **Direct Source**: Several direct campaigns have shown exceptionally high ROI with one instance reaching a 12.53 ROI on an ad spend of $330,944 generating over $4.1 million in revenue.

2. **Organic Efforts**:
   - **Google Organic**: With a notable ROI of 3.93 at an ad spend of $357,920 generating $1,406,743, focusing on organic search efforts may bring scalable growth without direct ad costs.
   - **Facebook Organic**: The organic source on Face